In [2]:
import cv2 as cv
import numpy as np

In [3]:
face_c = cv.CascadeClassifier("haarcascade_frontalface_default.xml")

In [18]:
def detect_face():
    cap = cv.VideoCapture("male_sample.mp4")

    # Define the codec and create VideoWriter object
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter('output.mp4',fourcc, 12.0, (768,432))

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret is True:
            gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            faces = face_c.detectMultiScale(gray)
            gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            faces = face_c.detectMultiScale(gray)
            for(x,y,w,h) in faces:
                cv.rectangle(frame,(x,y), (x+w, y+h), (255,0,0), 2)
                roi_gray = gray[y:y+h, x:x+w]
                roi_color = frame[y:y+h, x:x+w]
            #print "found " +str(len(faces)) +" face(s)"
            cv.imshow('frame', frame)
            out.write(frame)
            k = cv.waitKey(30) & 0xff
            if k == 27:
                break
        else:
            break

    # Release everything if job is finished
    cap.release()
    out.release()
    cv.destroyAllWindows()

In [30]:
def record_new_user(ident):
    """Open the default camera and record detected faces as images and labels them with an the given ID. To be used for the recognizer."""
    cap = cv.VideoCapture(0)
    id = 2
    sampleN=0;
    while 1:
        ret, img = cap.read()
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        faces = face_c.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            sampleN=sampleN+1;
            #cv.imwrite("C:/Users/Alex-/OneDrive - Arcada/Projects/Year3/Datorseende/FaceDetection/faceData/User."+str(id)+ "." +str(sampleN)+ ".jpg", gray[y:y+h, x:x+w])
            cv.imwrite("faceData/User."+str(id)+ "." +str(sampleN)+ ".jpg", gray[y:y+h, x:x+w])
            cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            cv.waitKey(100)
        cv.imshow('img',img)
        cv.waitKey(1)
        if sampleN > 500:
            break
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    cap.release()

    cv.destroyAllWindows()

In [31]:
from PIL import Image # For face recognition we will the the LBPH Face Recognizer 
import os
def train_recognizer():
    """Train a LBPH face recognizer with the data created from record_new_user"""
    recognizer = cv.face.LBPHFaceRecognizer_create() # pip install opencv-contrib-python
    #path="C:/Users/Alex-/OneDrive - Arcada/Projects/Year3/Datorseende/FaceDetection/faceData/"
    path="faceData/"
    def getImagesWithID(path):
        imagePaths = [os.path.join(path, f) for f in os.listdir(path)]   
     # print image_path   
     #getImagesWithID(path)
        faces = []
        IDs = []
        for imagePath in imagePaths:      
      # Read the image and convert to grayscale
            facesImg = Image.open(imagePath).convert('L')
            faceNP = np.array(facesImg, 'uint8')
            # Get the label of the image
            ID= int(os.path.split(imagePath)[-1].split(".")[1])
             # Detect the face in the image
            faces.append(faceNP)
            IDs.append(ID)
            cv.imshow("Adding faces for traning",faceNP)
            cv.waitKey(10)
        return np.array(IDs), faces
    Ids,faces  = getImagesWithID(path)
    recognizer.train(faces,Ids)
    #recognizer.save("C:/Users/Alex-/OneDrive - Arcada/Projects/Year3/Datorseende/FaceDetection/faceREC/trainingdata.yml")
    recognizer.save("faceREC/trainingdata.yml")
    
    cv.destroyAllWindows()

In [29]:
def run_recognition(src=0):
    cap = cv.VideoCapture(src)
    rec = cv.face.LBPHFaceRecognizer_create()
    rec.read("C:/Users/Alex-/OneDrive - Arcada/Projects/Year3/Datorseende/FaceDetection/faceREC/trainingdata.yml")
    id=0

    font = cv.FONT_HERSHEY_COMPLEX_SMALL
    #font=cv.InitFont(cv.FONT_HERSHEY_COMPLEX_SMALL,5,1,0,4)
    while 1:
        ret, img = cap.read()
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        faces = face_c.detectMultiScale(gray, 1.5, 5)
        for (x,y,w,h) in faces:
            cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            id,conf=rec.predict(gray[y:y+h,x:x+w])
            print(conf)
            confPrecision = 60
            if id==1 and conf < confPrecision:
                id="Alexander with glasses"
            elif id==2 and conf < confPrecision:
                id="Alexander no glasses"
            else:
                id="Unknown"
            cv.putText(img,str(id),(x,y+h),font,1, 255)
        cv.imshow('img',img)

        if cv.waitKey(1) == ord('q'):
            break
    cap.release()

    cv.destroyAllWindows()

In [27]:
def record_clip():
    cap = cv.VideoCapture(0)
    # Define the codec and create VideoWriter object
    fourcc = cv.VideoWriter_fourcc(*'DIVX')
    out = cv.VideoWriter('output.avi',fourcc, 20.0, (640,480))

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret is True:
            cv.imshow('frame', frame)
            out.write(frame)
            k = cv.waitKey(30) & 0xff
            if k == 27:
                break
        else:
            break

    # Release everything if job is finished
    cap.release()
    out.release()
    cv.destroyAllWindows()

In [22]:
record_clip()

In [10]:
detect_face()

In [12]:
record_new_user(2)

In [ ]:
train_recognizer()

In [28]:
run_recognition(0)

58.00788302155481
56.81277104262213
55.77127544239233
51.86787400309952
51.57207627754083
56.32566342552738
54.90414415455597
57.85978976544907
61.265792780576916
54.13373278817538
51.04281323374495
51.067027155833536
64.62599194111945
64.04243967974729
61.790057908441334
64.47639711072613
63.05337319226293
62.262177923213166
52.55437543849987
50.6420957026843
55.269397204781285
49.757002966445235
49.96885543225844
49.765854895416645
48.41639147880465
49.918869370917484
49.05787899584211
50.41559639087068
50.5418753775496
49.68196808830525
50.15730503555196
56.4840088764877
50.25704801689233
48.593776692186324
49.68777512651737
58.40457469154998
54.03650860300986
52.47223369179082
53.667628580795004
51.697092486575535
52.0001117838132
50.618355065102165
49.351594793972204
48.08293604311571
48.72883049961303
48.27184317678324
49.849888816037954
48.127786233652834
49.064357356096586
49.41884776953027
48.98011424256861
53.6129783111055
49.38809845287475
49.394992076404584
49.4389584616383

In [25]:
run_recognition("output.avi")

68.3492994582678
61.06611322603183
61.139257487135936
59.09145517691163
59.17530785592343
63.65469468891751
62.05209626562106
57.88166433237522
58.17310139311435
59.5171212244132
58.50803801822047
68.44134896285061
60.84767912214493
59.6984728330761
61.234337858433115
60.757053766934895
57.67647191092849
57.764605366823105
58.14693632616161
59.96281801140949
62.504282303255316
62.82573323841349
63.160249336932985
70.9955601804264
69.62544246166583
66.57001841446565
66.73207765424512
66.49867875816112
65.27883769430275
62.76185139006703
62.11929938538359
62.58327636388857
66.353235611298
66.3814446447604
67.0354321335253
67.5644529882067
68.26482129621083
64.06343537838372
63.69339798770377
64.24930255631178
62.89316045623116
65.27199516488476
61.0665375597764
61.77323887637926
61.01911234308981
59.40546645040073
59.567463575267766
60.86369848238872
60.73881001119623
62.03903049786567
71.23161467096058
64.52659726137797
56.95013574773268
57.666181440527474
56.57006110995758
55.237446235

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
